1. 程序集: 经编译器编译得到的,供进一步编译执行的中间产物,在Windows系统中,它一般表现的后缀是.dll(库文件),或者exe文件
2. 元数据: 用来描述数据的数据,有关程序以及类型的数据被称为元数据,他们保存在程序集中
3. 反射: 程序在运行时可以查看其他程序集或者自己的元数据(可以得到自己程序集代码的何种信息)
4. 反射的作用: 程序运行时得到所有的元数据; 程序运行中实例化对象, 程序运行时创建新的对象, 用这些对象执行任务(一个程序中使用另一个程序中的代码)

In [23]:
class ReflectExample
{
    private string message = "Hello from ReflectExample!";
    public void DisplayMessage()
    {
        Console.WriteLine("Id: {0}, Message: {1}", this.testId, this.message);
    }

    public int testId = 0;

    public ReflectExample()
    {
        this.testId = 0;
        this.message="Hello from ReflectExample!";
    }

    public ReflectExample(int id)
    {
        this.testId = id;
    }

    public ReflectExample(int id, string message):this(id)
    {
        this.message = message;
    }
}

1. Type类: 类的成员变量,成员函数等信息

In [24]:
using System.Reflection;
// Type类(类的信息类) 它是反射功能的基础 它时访问元数据的主要方式. 使用type成员获取有关类型声明的信息
int number = 10;
Type type = number.GetType();
Console.WriteLine(type);
Console.WriteLine(typeof(int));
Console.WriteLine(Type.GetType("System.Int32")); //类名必须包含命名空间
//得到类的程序集信息
Console.WriteLine(type.Assembly);
Console.WriteLine("===========================================");

//获取类的所有公共成员
Type typeExample = typeof(ReflectExample);
MemberInfo[] members = typeExample.GetMembers();
foreach (MemberInfo member in members)
{
    Console.WriteLine(member);
}

Console.WriteLine("===========================================");
//获取类的公共构造函数并调用
ConstructorInfo[] constructors = typeExample.GetConstructors();
foreach (ConstructorInfo constructor in constructors)
{
    Console.WriteLine(constructor);
}
//获取其中一个构造函数并执行,得到构造函数需要传入Type数组, 数组中的内容按顺序是参数类型
ConstructorInfo constructor = typeExample.GetConstructor(new Type[0]);
ReflectExample example = constructor.Invoke(null) as ReflectExample; //调用无参构造函数,没有参数写null
example.DisplayMessage();

//获取有参构造函数
constructor = typeExample.GetConstructor(new Type[] { typeof(int), typeof(string) });
example = constructor.Invoke(new object[] { 42, "Hello, Reflection!" }) as ReflectExample; //调用有参构造函数
example.DisplayMessage();

Console.WriteLine("===========================================");

//获取类的所有公共成员变量
FieldInfo[] fields = typeExample.GetFields();
foreach (FieldInfo field in fields)
{
    Console.WriteLine(field);
}

//获取指定名称的成员变量
FieldInfo fieldInfo = typeExample.GetField("testId");
if (fieldInfo != null)
{
    Console.WriteLine(fieldInfo);
} 

//通过反射获取和设置对象的值
Console.WriteLine(fieldInfo.GetValue(example));
fieldInfo.SetValue(example, 100);
Console.WriteLine(fieldInfo.GetValue(example));

Console.WriteLine("===========================================");

//获取类的所有公共方法
MethodInfo[] methods = typeExample.GetMethods();
foreach (MethodInfo method in methods)
{
    Console.WriteLine(method);
}

//获取类的特定的方法,重载函数需要第二个参数传入Type类型,没有重载则直接使用这个函数即可
MethodInfo specificMethod = typeExample.GetMethod("DisplayMessage", new Type[0]);
if (specificMethod != null)
{
    Console.WriteLine(specificMethod);
}

//调用特定的方法,传入类,和需要的参数, 参数传递为new object[] { }
//静态方法Invoke时第一个参数为null,实例方法则传入实例对象
specificMethod.Invoke(example, null); //无参数调用

System.Int32
System.Int32
System.Int32
System.Private.CoreLib, Version=9.0.0.0, Culture=neutral, PublicKeyToken=7cec85d7bea7798e
Void DisplayMessage()
System.Type GetType()
System.String ToString()
Boolean Equals(System.Object)
Int32 GetHashCode()
Void .ctor()
Void .ctor(Int32)
Void .ctor(Int32, System.String)
Int32 testId
Void .ctor()
Void .ctor(Int32)
Void .ctor(Int32, System.String)
Id: 0, Message: Hello from ReflectExample!
Id: 42, Message: Hello, Reflection!
Int32 testId
Int32 testId
42
100
Void DisplayMessage()
System.Type GetType()
System.String ToString()
Boolean Equals(System.Object)
Int32 GetHashCode()
Void DisplayMessage()
Id: 100, Message: Hello, Reflection!


2. Activator类 用于快速实例化对象的类,将Type对象快速实例化为对象

In [25]:
// 先得到type 然后快速实例化一个对象

class Test {
    public int i = 0;
    public string str = "*"; 
    public Test () {
        
    }
    public Test (int i) {
        this.i = i;
    }
    public Test (int i, string str):this(i) {
        this.str = str;
    }
}

Type test = typeof(Test);
//无参构造
Test testobj = Activator.CreateInstance(test) as Test; //默认使用默认构造函数
Console.WriteLine(testobj.str);
//有参构造
testobj = Activator.CreateInstance(test,99) as Test; //调用只有一个参数的构造函数
Console.WriteLine(testobj.i);
testobj = Activator.CreateInstance(test,100,"HelloWorld") as Test;
Console.WriteLine(testobj.i + " " + testobj.str);

*
99
100 HelloWorld


3. Assembly类:主要用于加载其他程序集,加载后才能使用Type来使用其他程序集中的信息

In [26]:
// 如果想要使用不是自己程序集中的内容,需要先加载程序集. eg:dll文件

//三种加载程序集的函数
//1. 一般用来加载同一个文件下的其他程序集
//Assembly assembly = Assembly.Load("程序集名称");
//2. 一般用来加载不在同一个文件夹下的其他程序集
//Assembly assembly = Assembly.LoadFrom("包含程序集清单的文件名称或路径")
//Assembly assembly = Assembly.LoadFile("要加载的文件的完全限定路径")
//LoadFrom和LoadFile使用方式与作用相同

//先加载指定一个程序集
Assembly assembly = Assembly.LoadFrom(@"D:\CSharpCode\21.反射\test.dll");
//得到程序集的类型信息
Type[] types = assembly.GetTypes();
for(int i = 0; i< types.Length; i++) {
    Console.WriteLine(types[i]);
    //打印types[i]类的具体信息
    MemberInfo[] members = types[i].GetMembers();
    for(int j = 0; j< members.Length; j++)  {
        Console.WriteLine(members[j]);
    }
    //实例化一个对象
    object obj = Activator.CreateInstance(types[i]);
    
    //调用方法
    MethodInfo move = types[i].GetMethod("Move");

    move.Invoke(obj,null); //Move方法没有参数 
    Console.WriteLine("==============================");
}

Error: System.BadImageFormatException: Bad IL format. The format of the file 'D:\CSharpCode\21.反射\test.dll' is invalid.
   at System.Runtime.Loader.AssemblyLoadContext.LoadFromAssemblyPath(String assemblyPath)
   at System.Reflection.Assembly.LoadFrom(String assemblyFile)
   at Submission#26.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)